In [1]:
import numpy as np
import pandas as pd
# Force to display all columns in the notebook
pd.set_option('display.max_columns', None)

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import r2_score

import plotly.express as px
import plotly.graph_objects as go

In [5]:
df = pd.read_csv("handing.csv")
df.columns = [name.strip() for name in df.columns]
df = df.sample(1000, random_state = 1)


In [8]:
df = df.drop('Unnamed: 0', axis=1)

In [9]:
df

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_max_shares,self_reference_avg_sharess,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,rate_positive_words,rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
507,http://mashable.com/2014/03/01/uk-spy-agency-y...,313.0,10.0,808.0,0.526448,1.0,0.716667,9.0,5.0,5.0,1.0,4.475248,9.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.0,678.0,190.571429,0.0,843300.0,146588.888889,0.000000,6342.000000,2922.248006,802.0,5200.0,1857.400000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.022820,0.665970,0.139121,0.149453,0.022637,0.480691,0.102104,0.061881,0.042079,0.595238,0.404762,0.432169,0.100000,0.8,-0.323162,-0.750000,-0.050000,0.900000,0.600000,0.400000,0.600000,14000
818,http://mashable.com/2013/11/30/beard-facts-comic/,404.0,8.0,148.0,0.688889,1.0,0.739583,3.0,3.0,1.0,12.0,4.594595,7.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,1900.0,565.000000,33400.0,843300.0,296671.428571,2814.814159,14433.333333,6207.207890,1900.0,1900.0,1900.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.354260,0.028606,0.028622,0.559770,0.028742,0.408889,-0.064062,0.020270,0.067568,0.230769,0.769231,0.166667,0.100000,0.2,-0.166667,-0.166667,-0.166667,0.433333,-0.166667,0.066667,0.166667,824
452,http://mashable.com/2013/03/26/t-mobile-iphone...,653.0,14.0,186.0,0.697297,1.0,0.834783,1.0,1.0,0.0,2.0,4.876344,4.0,0.0,0.0,1.0,0.0,0.0,0.0,217.0,469.0,355.750000,1600.0,617900.0,207300.000000,853.000000,3323.873016,2502.582700,4300.0,4300.0,4300.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.798945,0.050000,0.050000,0.050000,0.051054,0.301461,0.079545,0.026882,0.005376,0.833333,0.166667,0.247273,0.100000,0.5,-0.400000,-0.400000,-0.400000,0.000000,0.000000,0.500000,0.000000,794
368,http://mashable.com/2013/01/18/instagram-chang...,720.0,11.0,332.0,0.560976,1.0,0.692308,8.0,7.0,1.0,0.0,4.783133,5.0,0.0,0.0,0.0,1.0,0.0,0.0,217.0,1200.0,633.000000,4900.0,51900.0,28800.000000,2294.507246,5048.583333,3441.820170,1400.0,22300.0,6220.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.703196,0.040011,0.040287,0.040706,0.175800,0.517204,0.161156,0.045181,0.024096,0.652174,0.347826,0.400833,0.033333,0.6,-0.139583,-0.200000,-0.100000,0.000000,0.000000,0.500000,0.000000,3400
242,http://mashable.com/2013/06/19/miami-heat-fans/,568.0,14.0,183.0,0.644578,1.0,0.784314,3.0,0.0,0.0,1.0,4.103825,7.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,1200.0,453.142857,1000.0,690400.0,157942.857143,1000.000000,10687.488889,5521.891006,0.0,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.028572,0.028968,0.028572,0.885317,0.028572,0.738451,0.258535,0.065574,0.010929,0.857143,0.142857,0.375253,0.050000,0.7,-0.312500,-0.500000,-0.125000,0.727273,-0.181818,0.227273,0.181818,11000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
767,http://mashable.com/2013/11/13/icloud-producti...,421.0,7.0,594.0,0.523478,1.0,0.665782,41.0,2.0,18.0,0.0,5.193603,10.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,311.0,157.800000,30900.0,843300.0,183520.000000,3317.393223,5368.234358,3924.013009,3300.0,3300.0,3300.000000,0.0,0.0,1.0,0.0,0.0,0

In [11]:
df.info()
df.describe(include="all")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 507 to 37
Data columns (total 61 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   url                            1000 non-null   object 
 1   timedelta                      1000 non-null   float64
 2   n_tokens_title                 1000 non-null   float64
 3   n_tokens_content               1000 non-null   float64
 4   n_unique_tokens                1000 non-null   float64
 5   n_non_stop_words               1000 non-null   float64
 6   n_non_stop_unique_tokens       1000 non-null   float64
 7   num_hrefs                      1000 non-null   float64
 8   num_self_hrefs                 1000 non-null   float64
 9   num_imgs                       1000 non-null   float64
 10  num_videos                     1000 non-null   float64
 11  average_token_length           1000 non-null   float64
 12  num_keywords                   1000 non-null   f

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_max_shares,self_reference_avg_sharess,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,rate_positive_words,rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
count,1000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.00000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
unique,1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,http://mashable.com/2014/03/01/uk-spy-agency-y...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,350.293000,10.484000,540.058000,0.533191,0.976000,0.676836,10.799000,3.19400,4.570000,1.24000,4.567762,7.206000,0.044000,0.186000,0.157000,0.052000,0.173000,0.218000,25.368000,1254.721770,325.257962,14428.739000,758316.500000,262850.186592,1103.949210,6067.834469,3206.275802,4118.742000,10103.527000,6426.802485,0.156000,0.20800,0.176000,0.18000,0.15000,0.060000,0.070000,0.130000,0.184618,0.150211,0.214289,0.235232,0.215650,0.448697,0.121016,0.040191,0.017116,0.682238,0.293762,0.359513,0.100505,0.764513,-0.254494,-0.510839,-0.102380,0.277957,0.069438,0.353569,0.152021,3445.639000
std,NaN,212.748695,2.092396,465.540633,0.129250,0.153126,0.144861,11.115402,3.39085,8.918556,4.55754,0.767052,1.882314,0.205198,0.389301,0.363983,0.222138,0.378437,0.413094,68.809704,4912.033969,814.903741,62271.147325,205613.603713,135270.402539,1134.930216,8442.567738,1542.816273,21176.366006,34783.527985,22978.885021,0.363037,0.40608,0.381011,0.38438,0.35725,0.237606,0.255275,0.336472,0.261574,0.228773,0.274843,0.301203,0.280740,0.112302,0.096882,0.017710,0.011203,0.183592,0.156119,0.104204,0.078553,0.239552,0.119911,0.285152,0.083702,0.331816,0.268229,0.185251,0.231603,9484.595919
min,NaN,9.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.